## Homework 5 - Databases

Schema description:

table top airports:
aid primary key
icao secondary key
enplanements, ect
city secondary key

table airport info:
icao main key
long
lat
elevation
/////

table weather info:
wid primary key unique ect
city string
date Date
min temp Float
max temp
humid
precip


one city might apply to several airports

table airports:
aid primary key 
icao
city


QS: do I need a relationship?

In [29]:
import pandas as pd
from sqlalchemy import *#Table, MetaData, create_engine
from bs4 import BeautifulSoup
import requests
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, sessionmaker
import datetime
import time
from multiprocessing import Pool
import numpy as np

In [30]:
engine = create_engine('sqlite:///:memory:')
metadata = MetaData()
Base = declarative_base()

In [31]:
#create my table classes

class Airport(Base):
    __tablename__ = 'airports'
    id = Column(Integer, primary_key=True)
    ICAO = Column(String)
    city = Column(String)
    iso_region = Column(String)
    enplanements = Column(Integer)
    elevation_ft = Column(Float)
    latitude_deg = Column(Float)
    longitude_deg = Column(Float)
    weathers = relationship('AirportWeather', backref = 'Airport')
    
class AirportWeather(Base):
    __tablename__ = 'weathers'
    id = Column(Integer, primary_key=True)
    day = Column(Date)
    high = Column(Float)
    low = Column(Float)
    humidity = Column(Float)
    precipitation = Column(Float)
    events = Column(String)
    aid = Column(Integer, ForeignKey('airports.id'))
    

In [32]:
Base.metadata.create_all(engine) 


Session = sessionmaker(bind=engine)
session = Session()

In [33]:
top_airports_filename = "hw_5_data/top_airports.csv"
airport_info_filename = "hw_5_data/ICAO_airports.csv"

top_airports = pd.read_csv(top_airports_filename)
airport_info = pd.read_csv(airport_info_filename)

In [34]:
#homework suggests a single table for airports, so use pandas to merge these two tables
mrgd = top_airports.merge(airport_info, how='left',left_on = 'ICAO', right_on = 'ident')
#fix any nulls
mrgd = mrgd.where(pd.notnull(mrgd), None)

In [35]:
#so then populate an airport table
for i,row in mrgd.iterrows():
    a = Airport(city = row.City, ICAO=row.ICAO, iso_region = row.iso_region, enplanements = row.Enplanements,
                elevation_ft = row.elevation_ft,
               longitude_deg = row.longitude_deg, latitude_deg = row.latitude_deg)
    session.add(a)
#then commit to the session
session.commit()

a new version of these fxns to handle missing dates without hardcoding

!!!!!!!!! you could also just get that each row is a tbody, and you could maybe have neater results then?

In [40]:
def is_td_has_no_class(tag):
    """this is a function used to search the soup. All the items I want are named td but
    don't have a class definition, this will return True for them"""
    return tag.name =='td' and not tag.has_attr('class')

def add_weather_for_airport(a):
    t=time.time()
    months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
              'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    print(a.ICAO)
    ICAO = a.ICAO
    #loop thru years 2008-2018
    #I go year by year bc had issues requesting more than one year of info from the site
    for curr_year in range(2008,2019):
        
        #if the year is 2018, get info through the current day
        if curr_year == 2018:
            weather_req = requests.get(
                "https://www.wunderground.com/history/airport/" + ICAO + \
                '/' +str(curr_year)+ '/1/1/CustomHistory.html?dayend=6&monthend=3&yearend='
                + str(curr_year))
        #otherwise get it through the end of the year
        else:
            weather_req = requests.get(
                "https://www.wunderground.com/history/airport/" + ICAO + \
                '/' +str(curr_year)+ '/1/1/CustomHistory.html?dayend=31&monthend=12&yearend='
                +str(curr_year))
            
        soup = BeautifulSoup(weather_req.text,"html.parser")
        #the desired table has id obsTable. Within that, entries like 'high' and 'Temperature'
        #ect have a class associated with them, while all values don't have a class, so grab
        #all the tags with name td but that don't have a class
        table_entries=soup.find('table', id = 'obsTable').find_all(is_td_has_no_class)
        #then convert from these tag objects to the strings inside.
        #if there is more than one thing in the contents, the middle one is the one we want
        table_entries_str = [str(entry.text) if len(entry.contents)==1 \
                             else str(entry.contents[1].text) \
                             for entry in table_entries]
        #table_entries_str now contains only the rows containing info, except that there
        #is also a string indicating the month every so often. Other than this, think
        #of it as a table with 21 entries per row, but every so often there's a month
        #val that isn't a part of a row
        curr_month = 0
        curr_ind = 0
        ncols = 21
        while curr_ind < len(table_entries_str)-1:
            #first check if theres a month indicator
            if table_entries_str[curr_ind] in months:
                curr_month += 1 #if it was in months, increment months
                curr_ind += 1 #don't include month values in rows

            row = table_entries_str[curr_ind:curr_ind+ncols]
            
            #now prepare the data for entry into the database
            #clean up some weird html stuff
            if '\t' in row[20]:
                row[20] = ''
            else:
                row[20] = row[20].replace('\n','')

            #create a datetime.date object for the date
            dateday = datetime.date(int(curr_year), curr_month, int(row[0]))
            
            #sometimes the precipitation is 'T', count that as being none
            if row[19]=='T':
                row[19]=None
            
            #otherwise, if the value is '-', its missing and thus None
            for i,s in enumerate(row):
                if s is not None:
                    if '-' in s:
                        row[i]=None
            
            aw=AirportWeather(day = dateday, high = row[1], low = row[3], humidity = row[8],
                              precipitation =  row[19], events = row[20], aid = a.id)
            session.add(aw)
            curr_ind += ncols #increment curr_ind
    print('Time elapsed ' + str(time.time()-t))

In [41]:
session.rollback()

In [42]:
airport_list = session.query(Airport).all()
pool = Pool(processes = 2)
pool.map(add_weather_for_airport, airport_list[0:1])
#session.commit()
pool.close()
#session.rollback()


KATL
and now KATL
366
and now KATL
731
and now KATL
1096
and now KATL
1461
and now KATL
1827
and now KATL
2192
and now KATL
2557
and now KATL
2922
and now KATL
3288
and now KATL
3653
and now KATL
3718
Time elapsed 27.036519527435303


In [12]:
ny_airport_list = session.query(Airport).filter(Airport.city == 'New York').all()
pool = Pool(processes = 2)
pool.map(add_weather_for_airport, ny_airport_list)
pool.close()
session.commit()

KLGA
KJFK
Time elapsed 39.45159125328064


TypeError: must be str, not int

Process ForkPoolWorker-4:
Process ForkPoolWorker-3:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/hayley/.conda/envs/my_root/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/hayley/.conda/envs/my_root/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/hayley/.conda/envs/my_root/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/hayley/.conda/envs/my_root/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/hayley/.conda/envs/my_root/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/home/hayley/.conda/envs/my_root/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/home/hayley/.conda/envs/my_root/lib/python3.6/multiprocessing/queues.py", line 335, in get
    

In [18]:
print(len([aw.high for aw in session.query(AirportWeather).all()]))
print([a.id for a in ny_airport_list])

0
[6, 20]


In [ ]:
%matplotlib inline
#get the highs and the days for this airport
this_airport = [aw.high for aw in session.query(AirportWeather).filter(AirportWeather.aid==1).all()]
this_airport_dates = [aw.day for aw in session.query(AirportWeather).filter(AirportWeather.aid==1).all()]
this_airport = [aw.high for aw in session.query(AirportWeather).filter(AirportWeather.aid==2).all()]
this_airport_dates = [aw.day for aw in session.query(AirportWeather).filter(AirportWeather.aid==2).all()]
#[print(aw.day, aw.high) for aw in session.query(AirportWeather).filter(AirportWeather.aid==8).all()]
new_ind = pd.date_range('01-01-2008', '03-06-2018')
ser=pd.Series(this_airport, index = this_airport_dates)
ser=ser.reindex(new_ind)
ser.interpolate(method='time', inplace=True)
serdv = ser.diff()
print(serdv[0:10])
print(ser[0:10])
print(serdv[3700:])
print(ser[3700:])
print(len(serdv))
print(len(ser))
serdv[0:400].plot()
ser[0:400].plot()

the old version that hardcodes :(

In [15]:
def is_td_has_no_class(tag):
    """this is a function used to search the soup. All the items I want are named td but
    don't have a class definition, this will return True for them"""
    return tag.name =='td' and not tag.has_attr('class')

def add_weather_for_airport(a):
    t=time.time()
    months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    print(a.ICAO)
    ICAO = a.ICAO
    curr_year = 2008
    x=0
    for yr in range(0,11):
        
        if curr_year == 2018:
            weather_req = requests.get(
                "https://www.wunderground.com/history/airport/" + ICAO + \
                '/' +str(curr_year)+ '/1/1/CustomHistory.html?dayend=6&monthend=3&yearend='+str(curr_year))
        else:
            weather_req = requests.get(
                "https://www.wunderground.com/history/airport/" + ICAO + \
                '/' +str(curr_year)+ '/1/1/CustomHistory.html?dayend=31&monthend=12&yearend='+str(curr_year))
            
        soup = BeautifulSoup(weather_req.text,"html.parser")
        table_entries=soup.find('table', id = 'obsTable').find_all(is_td_has_no_class)
        table_entries_str = [str(entry.text) if len(entry.contents)==1 else str(entry.contents[1].text) for entry in table_entries]

    
        curr_month = 0
        curr_ind = 0
        ncols = 21
        if curr_year == 2018:
            dayrange = 31+28+6
        elif curr_year in [2008,2012, 2016]:
            dayrange = 366
        else:
            dayrange = 365
        for _ in range(0,dayrange):
            #for some crazy reason, KMSY is missing july 7th and 8th, so add those manually
            if ICAO=='KMSY':
                if _== 189 or _==190:
                    dateday = datetime.date(int(curr_year), curr_month, int(_-183))
                    aw=AirportWeather(day = dateday, high = None, low = None, humidity = None,
                              precipitation =  None, events = None, aid = a.id)
                    session.add(aw)
                    continue
            #first check if theres a month indicator
            if curr_ind > len(table_entries_str)-1:
                print('uh oh ' + ICAO + ' ' + str(curr_month) + ' ' + str(curr_year) + ' ' + str(_))
            if table_entries_str[curr_ind] in months:
                curr_month += 1
                x+=(_-x)
                curr_ind += 1

            row = table_entries_str[curr_ind:curr_ind+ncols]
            if '\t' in row[20]:
                row[20] = ''
            else:
                row[20] = row[20].replace('\n','')

            curr_ind += ncols
            dateday = datetime.date(int(curr_year), curr_month, int(row[0]))
            
            #sometimes the precipitation is just T? so handle that I guess
            if row[19]=='T':
                row[19]=None
                
            for i,s in enumerate(row):
                if s is not None:
                    if '-' in s:
                        row[i]=None
            aw=AirportWeather(day = dateday, high = row[1], low = row[3], humidity = row[8],
                              precipitation =  row[19], events = row[20], aid = a.id)
            session.add(aw)
        curr_year += 1
    print('Time elapsed ' + str(time.time()-t))


In [ ]:
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

for a in session.query(Airport)[4:5]:
    t=time.time()
    print(a.ICAO)
    ICAO = a.ICAO
    curr_year = 2008
    for yr in range(0,11):
        
        if curr_year == 2018:
            weather_req = requests.get(
                "https://www.wunderground.com/history/airport/" + ICAO + \
                '/' +str(curr_year)+ '/1/1/CustomHistory.html?dayend=6&monthend=3&yearend='+str(curr_year))
        else:
            weather_req = requests.get(
                "https://www.wunderground.com/history/airport/" + ICAO + \
                '/' +str(curr_year)+ '/1/1/CustomHistory.html?dayend=31&monthend=12&yearend='+str(curr_year))
            
        soup = BeautifulSoup(weather_req.text,"html.parser")
        table_entries=soup.find('table', id = 'obsTable').find_all(is_td_has_no_class)
        table_entries_str = [str(entry.text) if len(entry.contents)==1 else str(entry.contents[1].text) for entry in table_entries]

    
        curr_month = 0
        curr_ind = 0
        ncols = 21
        if curr_year == 2018:
            dayrange = 31+28+6
        elif curr_year in [2008,2012, 2016]:
            dayrange = 366
        else:
            dayrange = 365
        for _ in range(0,dayrange):
            #first check if theres a month indicator
            if table_entries_str[curr_ind] in months:
                curr_month += 1
                curr_ind += 1

            row = table_entries_str[curr_ind:curr_ind+ncols]
            if '\t' in row[20]:
                row[20] = ''
            else:
                row[20] = row[20].replace('\n','')

            curr_ind += ncols
            dateday = datetime.date(int(curr_year), curr_month, int(row[0]))
            
            #sometimes the precipitation is just T? so handle that I guess
            if row[19]=='T':
                row[19]=None
                
            for i,s in enumerate(row):
                if s is not None:
                    if '-' in s:
                        row[i]=None
            aw=AirportWeather(day = dateday, high = row[1], low = row[3], humidity = row[8],
                              precipitation =  row[19], events = row[20], aid = a.id)
            session.add(aw)
        print('Time elapsed ' + str(t-time.time()))
        session.commit()
        curr_year += 1

In [ ]:
#do pairwise correlations somehow
#store 4D matrix of the correlations for all 50
correlations = np.empty((6,6))
for a in session.query(Airport).all()[0:5]:
    this_airport = [aw.high for aw in session.query(AirportWeather).filter(AirportWeather.aid==a.id).all()]
    #need to confirm this is ordered somehow
    all_other_airports = [[aw.high for aw in
                           session.query(AirportWeather).filter(AirportWeather.aid==a2.id).all()] for a2 in
                         session.query(Airport).filter(Airport.id <=6).all()]
    
    x = pd.Series(this_airport
    y = [pd.Series(highs) for highs in all_other_airports]
    #fill nans, then take the derivative of each because its searching for change
    x.fillna(method='bfill', inplace=True)
    y = [ser.fillna(method='bfill') for ser in y]
    y = [ser.astype('float64', inplace=True) for ser in y]
    #take derivative bc searching for change
    xdv=x.diff() / x.index.to_series().diff()
    ydv=[y.diff() / y.index.to_series().diff()
    x = pd.Series(this_airport[0:3718-3])
    y = [pd.Series(highs[3:3718]) for highs in all_other_airports]
    x.fillna(method='bfill', inplace=True)
    y = [ser.fillna(method='bfill') for ser in y]
    y = [ser.astype('float64', inplace=True) for ser in y]
    x2 = x.values
    y2 = [ser.values.astype('float64') for ser in y]
    corr = np.corrcoef(x2,y2)
    print(np.max(corr[0,:]))
    correlations[(a.id-1),:] = corr[0,1:]
    #correlations(:,:,idy, a.id) = np.corrcoef()

In [ ]:
# x=np.unravel_index(np.argsort(correlations.ravel()), correlations.shape)
print(type(x))
x=np.asarray(x)
print(x[:,0])
print(x[:,360])
print(x[:,350:361])
print(session.query(Airport).filter(Airport.id==16).first().city)
print(session.query(Airport).filter(Airport.id==6).first().city)
print(correlations[0,:])
print(np.max(correlations))

In [ ]:
#so 0 and 6 are well linked
print(session.query(Airport).filter(Airport.id==6).first().ICAO)
session.query(Airport).filter(Airport.id==1).first().ICAO
session.query(Airport).filter(Airport.id==3).first().ICAO
this_airport = [aw.high for aw in session.query(AirportWeather).filter(AirportWeather.aid==1).all()]
jfk_airport = [aw.high for aw in session.query(AirportWeather).filter(AirportWeather.aid==6).all()]
lax_airport = [aw.high for aw in session.query(AirportWeather).filter(AirportWeather.aid==3).all()]
b_airport = [aw.high for aw in session.query(AirportWeather).filter(AirportWeather.aid==5).all()]

x = pd.Series(this_airport[0:3718-3])
y = pd.Series(jfk_airport[3:3718])
z = pd.Series(lax_airport[3:3718])
q = pd.Series(b_airport[3:3718])
x.fillna(method='bfill', inplace=True)
y.fillna(method='bfill', inplace=True)
z.fillna(method='bfill', inplace=True)
q.fillna(method='bfill', inplace=True)

import matplotlib.pyplot as plt
plt.subplot(1,3,1)
plt.scatter(x.values, y.values)
plt.subplot(1,3,2)
plt.scatter(x.values, z.values)
plt.subplot(1,3,3)
plt.scatter(x.values, q.values)
plt.gcf().set_size_inches(12,4)
plt.show()

In [ ]:
import numpy as np

things = [aw.high for aw in session.query(AirportWeather).filter(AirportWeather.aid==1).all()]
things2 = [aw.high for aw in session.query(AirportWeather).filter(AirportWeather.aid==2).all()]
#do the derivative because its change 
x = pd.Series(things[0:3718-3])
y = pd.Series(things2[3:3718])
x.fillna(method='bfill', inplace=True)
y.fillna(method='bfill', inplace=True)
y.astype('float64', inplace=True)
x2 = x.values
y2 = y.values
y2 = y2.astype('float64')
print(type(x.values))
print(type(y[1]))
print(type(x[1]))
print(np.corrcoef(x2,y2))
import matplotlib.pyplot as plt
plt.scatter(x2,y2)

In [ ]:
import numpy as np

things = [aw.high for aw in session.query(AirportWeather).filter(AirportWeather.aid==1).all()]
things2 = [aw.high for aw in session.query(AirportWeather).filter(AirportWeather.aid==2).all()]
#do the derivative because its change 
x = pd.Series(things[0:3718-3])
y = pd.Series(things2[3:3718])
x.fillna(method='bfill', inplace=True)
y.fillna(method='bfill', inplace=True)
y.astype('float64', inplace=True)
x2 = x.values
y2 = y.values
y2 = y2.astype('float64')
print(type(x.values))
print(type(y[1]))
print(type(x[1]))
print(np.corrcoef(x2,y2))
import matplotlib.pyplot as plt
plt.scatter(x2,y2)